In [19]:
import os
import cv2
import torch
import h5py
import numpy as np
import json
from tqdm import trange

In [20]:
repo_path = "/home/sjauhri/IAS_WS/EgoVis/2handedafforder_repo/"
dataset_folder = repo_path + "BENCHMARK/dataset_bench/EPIC_DATA/datasets/dset/P20_03/"
original_frame_folder = repo_path + "BENCHMARK/frames/P20/P20_03/"
full_inpainted_frame_folder = repo_path + "BENCHMARK/inpainted/hand_inpainting/EPIC_DATA/hand_inpainting/P20_03/"
out_folder = dataset_folder
visualization_active = True

In [ ]:
# loop through all the subdirectories in the data_folder provided
for subdir, dirs, files in os.walk(dataset_folder, topdown=True):

    print(f"remove color jitter folders...")
    # for dir in dirs:
    for i in trange(len(dirs)):
        dir = dirs[i]
        if 'cj' in dir:
            # remove
            os.system(f"rm -rf {subdir + dir}")

In [24]:
orginal_frame_file_prefix = "frame_000"

# loop through all the subdirectories in the data_folder provided
for subdir, dirs, files in os.walk(dataset_folder, topdown=True):

    print(f"Copying {len(dirs)} original frames to subdirectories in: {subdir}")
    # for dir in dirs:
    for i in trange(len(dirs)):
        dir = dirs[i]
        if 'cj' in dir:
            # skip
            continue
    
        # fetch the original video frame corresponding to this datapoint
        frame_id = dir
        # print("Frame ID: ", frame_id)
        original_frame_file_path = original_frame_folder + orginal_frame_file_prefix + frame_id + ".jpg"

        # copy origianl frame file to current subdirectory
        cmd = "cp " + original_frame_file_path + " " + subdir + dir + "/"
        # print(cmd)
        os.system(cmd)

        # print(f"Copied {original_frame_file_path} to {subdir + dir}/")

    break
    

Copying 378 original frames to subdirectories in: /home/sjauhri/IAS_WS/EgoVis/2handedafforder_repo/BENCHMARK/dataset_bench/EPIC_DATA/datasets/dset/P14_05/


100%|██████████| 378/378 [00:00<00:00, 583.29it/s]


In [11]:
# loop through all the subdirectories in the data_folder provided
for subdir, dirs, files in os.walk(dataset_folder, topdown=True):

    print(f"Copying {len(dirs)} full inpainted frames to subdirectories in: {subdir}")
    # for dir in dirs:
    for i in trange(len(dirs)):
        dir = dirs[i]
        if 'cj' in dir:
            # skip
            continue
    
        # fetch the full inpainted video frame corresponding to this datapoint
        frame_id = dir
        # print("Frame ID: ", frame_id)
        full_inpainted_frame_file_path = full_inpainted_frame_folder + frame_id + ".png"

        # copy full inpainted frame file to current subdirectory
        dest_file_path = subdir + dir + "/full_inpainted_frame.png"
        cmd = "cp " + full_inpainted_frame_file_path + " " + dest_file_path
        # print(cmd)
        os.system(cmd)

        # print(f"Copied {full_inpainted_frame_file_path} to {dest_file_path}")
        # break

    break

Copying 8982 full inpainted frames to subdirectories in: /home/sjauhri/IAS_WS/EgoVis/2handedafforder_repo/BENCHMARK/dataset_bench/EPIC_DATA/datasets/dset/P01_01/


100%|██████████| 8982/8982 [00:09<00:00, 918.21it/s] 


In [21]:
# Create viz images for annotation (affordance class + inpainted image + original image)

orginal_frame_file_prefix = "frame_000"

# loop through all the subdirectories in the data_folder provided
for subdir, dirs, files in os.walk(dataset_folder, topdown=True):

    print(f"Creating {len(dirs)} benchmark frames in subdirectories in: {subdir}")

    # for dir in dirs:
    for i in trange(len(dirs)):
        dir = dirs[i]
        if 'cj' in dir:
            # skip
            continue
        # dir = '0011885'
        # fetch the original video frame corresponding to this datapoint
        frame_id = dir
        # print("Frame ID: ", frame_id)
        original_frame_file_path = subdir + dir + "/" + orginal_frame_file_prefix + frame_id + ".jpg"

        # inpainted_frame_file_path = subdir + dir + "/inpainted_frame.png"
        inpainted_frame_file_path = subdir + dir + "/full_inpainted_frame.png"

        # get the affordance class from the json file
        affordance_annotation_file_path = subdir + dir + "/annotation.json"

        # create new image by concatenating the inpainted image, original image and affordance class
        original_frame = cv2.imread(original_frame_file_path)
        inpainted_frame = cv2.imread(inpainted_frame_file_path)
        inpainted_frame = cv2.cvtColor(inpainted_frame, cv2.COLOR_BGR2RGB)
        inpainted_frame = cv2.resize(inpainted_frame, (inpainted_frame.shape[1] * 2, inpainted_frame.shape[0] * 2))
        with open(affordance_annotation_file_path, 'r') as f:
            annotation_data = json.load(f)
            aff_class = annotation_data.get('narration', 'Unknown')
        if original_frame is not None and inpainted_frame is not None:
            # Scale down the original frame to 2/3rd the size
            original_frame = cv2.resize(original_frame, (original_frame.shape[1] * 2 // 3, original_frame.shape[0] * 2 // 3))

            # Resize original frame to match the height of the inpainted frame
            height_diff = inpainted_frame.shape[0] - original_frame.shape[0]
            black_space = np.zeros((height_diff, original_frame.shape[1], 3), dtype=np.uint8)

            # Add white text to the black space
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1
            font_color = (255, 255, 255)
            thickness = 2
            text = f"Task:"
            text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
            text_x = (black_space.shape[1] - text_size[0]) // 2
            text_y = (black_space.shape[0] + text_size[1]) // 2
            cv2.putText(black_space, text, (text_x, text_y), font, font_scale, font_color, thickness)
            text = f"{aff_class}"
            # Split the text into multiple lines if it doesn't fit in the width
            max_width = black_space.shape[1]
            words = text.split()
            lines = []
            current_line = words[0]

            for word in words[1:]:
                if cv2.getTextSize(current_line + ' ' + word, font, font_scale, thickness)[0][0] < max_width:
                    current_line += ' ' + word
                else:
                    lines.append(current_line)
                    current_line = word
            lines.append(current_line)

            y_offset = text_y + 50
            for line in lines:
                text_size = cv2.getTextSize(line, font, font_scale, thickness)[0]
                text_x = (black_space.shape[1] - text_size[0]) // 2
                cv2.putText(black_space, line, (text_x, y_offset), font, font_scale, font_color, thickness)
                y_offset += text_size[1] + 10

            original_frame = np.vstack((black_space, original_frame))

            benchmark_frame = np.concatenate((inpainted_frame, original_frame), axis=1)
            benchmark_frame_path = subdir + dir + "/benchmark_frame.png"
            cv2.imwrite(benchmark_frame_path, benchmark_frame)
            # print(f"Saved benchmark frame to {benchmark_frame_path}")

        else:
            raise Exception("Original or inpainted frame is not found")
        
    break

Creating 1087 benchmark frames in subdirectories in: /home/sjauhri/IAS_WS/EgoVis/2handedafforder_repo/BENCHMARK/dataset_bench/EPIC_DATA/datasets/dset/P20_03/


100%|██████████| 1087/1087 [00:09<00:00, 113.24it/s]
